In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "/home/snt/projects_lujun/base_models/gemma-2-2b-it"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id,  device_map={"":0})


Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.26s/it]


In [4]:
text = "Quote: Imagination is more"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=200)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


/home/snt/miniconda3/envs/unsloth_env/lib/python3.11/site-packages/torch/_inductor/compile_fx.py:194: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(
CUDAGraph supports dynamic shapes by recording a new graph for each distinct input size. Recording too many CUDAGraphs may lead to extra overhead. We have observed 51 distinct sizes. Please consider the following options for better performance: a) padding inputs to a few fixed number of shapes; or b) set torch._inductor.config.triton.cudagraph_skip_dynamic_graphs=True. Set torch._inductor.config.triton.cudagraph_dynamic_shape_warn_limit=None to silence this warning.


Quote: Imagination is more important than knowledge. For knowledge is limited, whereas imagination embraces the entire world, stimulating the mind and making it more creative.

This quote is a powerful statement about the importance of imagination in our lives. It highlights the limitations of knowledge and the boundless potential of imagination. 

Here's a breakdown of the quote and its implications:

**"Imagination is more important than knowledge."**

* This statement challenges the traditional view that knowledge is the most valuable asset. 
* It suggests that imagination is a more powerful force for change and innovation.
* Imagination allows us to envision possibilities that might not be possible with just knowledge.

**"For knowledge is limited, whereas imagination embraces the entire world."**

* This emphasizes the vastness and depth of imagination compared to the finite nature of knowledge.
* Knowledge is often confined to specific areas of expertise or information.
* Imagina

In [5]:
from datasets import load_dataset

data = load_dataset("Abirate/english_quotes")
data = data.map(lambda samples: tokenizer(samples["quote"]), batched=True)


Map: 100%|██████████| 2508/2508 [00:00<00:00, 9733.25 examples/s] 


In [ ]:
import transformers
from trl import SFTTrainer

def formatting_func(example):
    text = f"Quote: {example['quote'][0]}\nAuthor: {example['author'][0]}<eos>"
    return [text]

trainer = SFTTrainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=10,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    formatting_func=formatting_func,
)
trainer.train()
